# Продвинутый Python, Семинар 1

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

<div align="center">
    <img src="https://cdn-icons-png.flaticon.com/512/5301/5301145.png" height="128px" width="128px">
    <br>
    <i>Взаимодействие с ОС. Консольные утилиты.</i>
</div>

Прежде, чем начать обещанный рассказ про `argparse`, давайте еще немного углубимся в `pathlib`. На лекции мы изучили только основной функционал - на самом деле, модуль способен на большее.

**Пример.** Получить список объектов в директории.

In [ ]:
import pathlib


# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

# Просим список файлов
list(directory.iterdir())  # <-- Генератор

[WindowsPath('directory/file_1.txt'), WindowsPath('directory/file_2.txt')]

**Примечание.** Метод `iterdir` перебирает только те объекты, которые находятся на глубине `1` относительно корня.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

inner_directory = directory / "inner_directory"
inner_directory.mkdir(exist_ok=True)

file_3 = inner_directory / "file_3.txt"
file_3.touch(exist_ok=True)

# Вложенного файла нет в генераторе
assert file_3 not in directory.iterdir()

# Просим список файлов
list(directory.iterdir())

[WindowsPath('directory/file_1.txt'),
 WindowsPath('directory/file_2.txt'),
 WindowsPath('directory/inner_directory')]

**Задание.** Реализовать генератор, который обходит все файлы внутри переданной директории. Если встретилась символическая ссылка и пользователь выставил параметр `follow_symlinks`, то необходимо добавить замыкающий объект в перебор. Если символическая ссылка указывает на директорию, то добавляем ее файлы тоже. Напоминаем, что символическая ссылка может указывать на другую символическую ссылку.

In [ ]:
from collections.abc import Iterator


def expand(root: pathlib.Path, *, follow_symlinks: bool = False) -> Iterator[pathlib.Path]:
    raise NotImplementedError("Hello, Python")

In [ ]:
# Подчищаем для будущих примеров
# И вспоминаем, как удалять лишнее

# Удаляем регулярные файлы
for file in [file_1, file_2, file_3]:
    file.unlink()

# Должна быть пустой
inner_directory.rmdir()

**Пример.** Перебрать все объекты, которые заканчиваются на `.py` и которые находятся на глубине, равной единице.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.py"
file_2.touch(exist_ok=True)

# Ищем нужные файлы
for path in directory.iterdir():
    if path.suffix == ".py":
        print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_2.py


**Замечание.** Предложенный выше подход нельзя называть удобным. Интуитивно мы бы, скорее всего, хотели иметь что-то похожее на регулярные выражения, но для файловой системы. Для этого придумали `glob`-паттерны. Давайте сначала напишем код, а потом объясним, что произошло.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.py"
file_2.touch(exist_ok=True)

# Ищем нужные файлы
for path in directory.glob("*.py"):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_2.py


**Пояснение.** Маски `glob`-паттернов устроены не сложнее, чем регулярные выражения. Есть три вида масок: `*`, `?` и `[abc]`.

1. `*` - сопоставляет любую последовательность символов, включая пустоту;
2. `?` - сопоставляет ровно один символ;
3. `[abc]` - сопоставляет любой из символов `a`, `b` или `c` и только их.

**Пояснение.** В примере выше мы сказали, что нам нужно найти все объекты, в которых суффиксу `.py` предшествует последовательность символов любой длины.

**Пример.** Использование `?`.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

strange_file = directory / "file_.txt"  # без числа
strange_file.touch(exist_ok=True)

# Ищем нужные файлы
for path in directory.glob("file_?.txt"):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_1.txt
directory\file_2.txt


**Пример.** Использование `[...]`.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

file_3 = directory / "file_3.txt"
file_3.touch(exist_ok=True)

# Ищем нужные файлы
for path in directory.glob("file_[12].txt"):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_1.txt
directory\file_2.txt


**Замечание.** Маска `[...]` умеет делать еще кое-что: запрещать и сокращать. Объяснясем:

1. Маска вида `[!abc]` сопоставляет любой символ, кроме `a`, `b` или `c`;
2. Маска вида `[a-d]` сопоставляет любой символ, который входит в диапазон от `a` до `d` (`a`, `b`, `c` и `d`);
3. Маска вида `[!a-d]` сопоставляет любой символ, кроме тех, что входят в диапазон от `a` до `d`.

**Пример.** Использование `[!abc]`.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.txt"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

file_3 = directory / "file_3.txt"
file_3.touch(exist_ok=True)

file_4 = directory / "file_4.txt"
file_4.touch(exist_ok=True)

file_5 = directory / "file_5.txt"
file_5.touch(exist_ok=True)

# Ищем нужные файлы (удобнее, чем [345])
for path in directory.glob("file_[!12].txt"):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_3.txt
directory\file_4.txt
directory\file_5.txt


**Пример.** Использование `[a-d]`.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_a = directory / "file_a.txt"
file_a.touch(exist_ok=True)

file_b = directory / "file_b.txt"
file_b.touch(exist_ok=True)

file_c = directory / "file_c.txt"
file_c.touch(exist_ok=True)

file_d = directory / "file_d.txt"
file_d.touch(exist_ok=True)

file_e = directory / "file_e.txt"
file_e.touch(exist_ok=True)

# Ищем нужные файлы
for path in directory.glob("file_[b-d].txt"):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_b.txt
directory\file_c.txt
directory\file_d.txt


**Замечание.** Обратите внимание, что в силу синтаксических ограничений у Вас не получится написать очень сложных запросов. Например, если мы хотим отфильтровать файлы по нескольким расширениям, то, скорее всего, придется просто объединить несколько генераторов.

**Задание.** Найти все файлы с расширениями `.png` и `.mp4`.

In [ ]:
from itertools import chain


# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.png"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.py"
file_2.touch(exist_ok=True)

file_3 = directory / "file_3.png"
file_3.touch(exist_ok=True)

file_4 = directory / "file_4.mp4"
file_4.touch(exist_ok=True)

file_5 = directory / "file_5.hpp"
file_5.touch(exist_ok=True)

# Ищем нужные файлы
generators = [
    ## YOUR CODE
]

for path in chain.from_iterable(generators):
    print(path)

# Подчищаем для будущих примеров
for path in directory.iterdir():
    path.unlink()

directory\file_1.png
directory\file_3.png
directory\file_4.mp4


**Задача.** Найти все файлы с расширением `.py`.

In [ ]:
# Готовим файлы
directory = pathlib.Path("directory/")
directory.mkdir(exist_ok=True)

file_1 = directory / "file_1.py"
file_1.touch(exist_ok=True)

file_2 = directory / "file_2.txt"
file_2.touch(exist_ok=True)

inner_directory = directory / "inner_directory"
inner_directory.mkdir(exist_ok=True)

file_3 = inner_directory / "file_3.py"
file_3.touch(exist_ok=True)

# Просим список файлов
## YOUR CODE

[WindowsPath('directory/file_1.py')]

**Замечание.** На наше удивление, мы не нашли `inner_directory/file_3.py` в списке. Дело в том, что `glob`-паттерны раскрывают `*` только на уровне компонент. Слеши в `*` не входят. Если мы хотим добавить вложенности, то понадобится другая маска - `**`.

**Пояснение.** Маска `**` сопоставляет любое количество символов, включая слеши.

In [ ]:
list(directory.glob("**/file_?.py"))

[WindowsPath('directory/file_1.py'),
 WindowsPath('directory/inner_directory/file_3.py')]

In [ ]:
list(directory.glob("inner_directory/**/file_?.py"))

[WindowsPath('directory/inner_directory/file_3.py')]

Если мы хотим добавить `**` в начало, чтобы найти все-все-все объекты вне зависимости от их вложенности, то вместо `glob` стоит воспользоваться `rglob`.

In [ ]:
list(directory.rglob("file_?.py"))

[WindowsPath('directory/file_1.py'),
 WindowsPath('directory/inner_directory/file_3.py')]

**Пояснение.** Семантически `rglob` - это то же, что и `glob`, но с добавленным в начало `**/`. Такой вот синтаксический сахар.

In [ ]:
list(directory.glob("**/file_?.py"))

[WindowsPath('directory/file_1.py'),
 WindowsPath('directory/inner_directory/file_3.py')]

In [ ]:
list(directory.rglob("file_?.py"))

[WindowsPath('directory/file_1.py'),
 WindowsPath('directory/inner_directory/file_3.py')]

**Пример.** Получить текущую директорию.

In [ ]:
pathlib.Path.cwd()

WindowsPath('c:/Users/FCOla/Desktop')

**Замечание.** Вместо `pathlib.Path(".")` настоятельно рекомендуется использовать конструктор `pathlib.Path.cwd`.

**Пример.** Получить домашнюю директорию пользователя.

In [ ]:
pathlib.Path.home()

WindowsPath('C:/Users/FCOla')

**Пример.** Раскрыть домашнюю директорию в полный путь.

In [ ]:
path = pathlib.Path("~/Desktop")
path.expanduser()

WindowsPath('C:/Users/FCOla/Desktop')

**Замечание.** Обратите внимание, что знакомый Вам метод `absolute` не подойдет для этой цели.

In [ ]:
path = pathlib.Path("~/Desktop")
path.absolute()

WindowsPath('c:/Users/FCOla/Desktop/~/Desktop')

Теперь перейдем к разработке консольных утилит. В частности, нас будет интересовать модуль стандартной библиотеки, который мы немного задели на лекции, а именно `argparse`. Он предназначен для двух целей: обрабатывать пользовательский ввод (на уровне аргументов командной строки) и формирование самого интерфейса на основе указанной спецификации.

На всякий случай, кратенько напомним, как выглядят консольные утилиты. Вы с ними наверняка сталкивались, когда использовали, например, `ls` или `cat`. Собственно, это и есть приложения с консольным интерфейсом. Ниже приведем еще парочку примеров.

**Пример 1.**

```console
$ ls
a.txt b.txt c.txt
```

**Пример 2.**

```console
$ cat a.txt
Hello, World
```

**Пример 3.**

```console
$ python -B -m main
Run the code...
```

**Пример 4.**

```console
$ git add app/presentation/api/
$ git commit -m "feat: add the auth endpoint" -n
```

**Пример 5.**

```console
$ tree -cDr ./dev/
[Dec 31 23:55]  dev/
|-- [Dec 31 23:59] file.txt
```

**Пример 6.**

```console
$ git rebase --help
...
```

**Пример.** Реализуем аналог `cat`.

In [ ]:
import argparse


def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        prog="cat",
        description="Hello, File!",
        epilog="by @syubogdanov",
    )

    parser.add_argument("path")

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    args = parse_args(argv)

    path = pathlib.Path(args.path)
    print(path.read_text(errors="ignore"))


if __name__ == "__main__":
    file = pathlib.Path("file.txt")
    file.write_text("Hello, World")

    main([file.as_posix()])

Hello, World


**Пояснение.** Как правило, обработку аргументов командной строки выделяют в отдельную верхнеуровневую функцию. В нашем случае, ей выступает `parse_args`. В ней мы инициализируем обработчик (парсер), у которого определяем имя программы, описание и эпилог. Заставялем парсер принимать один *обязательный* аргумент, чье имя `path`. Дальше парсим переданные аргументы - и получаем пространство имен (`Namespace`). В нем и находятся обработанные аргументы.

In [ ]:
parse_args(["/home/dev/some/path"])

Namespace(path='/home/dev/some/path')

**Примечание.** Если не передавать аргументы в `parser.parse_args`, то по умолчанию будет использоваться `sys.argv`, то есть те аргументы, которые Вы передавали программе извне.

**Замечание.** Помимо обработки аргументов, модуль `argparse` также создает полноценный интерфейс. Давайте запросим подсказку по утилите (кстати, тоже добавляется автоматически).

In [ ]:
main(["--help"])

usage: cat [-h] path

Hello, File!

positional arguments:
  path

options:
  -h, --help  show this help message and exit

by @syubogdanov


SystemExit: 0

C:\Users\FCOla\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**Замечание.** Аргумент `--help`, или сокращенно `-h`, является опциональным аргументом, так как начинается с `--` (`-`). Это, как понятно из названия, значит, что нам не обязательно указывать аргмент, чтобы программа работала, но при этом ничего не мешает его активировать, так как он добавляет какую-то логику в поведение программы.

**Пример.** Добавим возможность указать число символов, которые необходимо вывести.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(prog="cat")

    parser.add_argument("path")

    parser.add_argument("-s", "--size")

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    args = parse_args(argv)

    path = pathlib.Path(args.path)
    text = path.read_text(errors="ignore")

    if args.size is not None:
        size = int(args.size)
        text = text[:size]

    print(text)


if __name__ == "__main__":
    file = pathlib.Path("file.txt")
    file.write_text("Hello, World")

    main([file.as_posix(), "--size", "5"])

Hello


In [ ]:
main([file.as_posix()])

Hello, World


In [ ]:
parse_args([file.as_posix(), "--size", "5"])

Namespace(path='file.txt', size='5')

**Замечание.** Ничего не мешает нам сломать программу. Например, передать вместо числа какую-то строчку (попробуйте сами). Давайте исправим это. Сразу зададим тип, который нам нужен. Получим двойную выгоду - добавим валидацию, а в пространство имен попадет сконвертированный тип.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(prog="cat")

    parser.add_argument("path", type=pathlib.Path)

    parser.add_argument("-s", "--size", type=int)

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    args = parse_args(argv)

    text = args.path.read_text(errors="ignore")

    if args.size is not None:
        text = text[:args.size]

    print(text)


if __name__ == "__main__":
    file = pathlib.Path("file.txt")
    file.write_text("Hello, World")

    main([file.as_posix(), "--size", "5"])

Hello


In [ ]:
parse_args([file.as_posix(), "--size", "5"])

Namespace(path=WindowsPath('file.txt'), size=5)

**Пример.** Добавим больше пользовательского текста в автосгенерированную подсказку.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(prog="cat")

    parser.add_argument(
        "path",
        help="the path to the file",
        metavar="PATH",
        type=pathlib.Path
    )

    parser.add_argument(
        "-s",
        "--size",
        help="the size of the text to be output",
        metavar="SIZE",
        type=int
    )

    return parser.parse_args(args)

In [ ]:
main(["--help"])

usage: cat [-h] [-s SIZE] PATH

positional arguments:
  PATH                  the path to the file

options:
  -h, --help            show this help message and exit
  -s SIZE, --size SIZE  the size of the text to be output


SystemExit: 0

**Пояснение.** Параметр `help` добавляет соответствующий текст-подсказку к аргументу, а `metavar` переобозначает аргумент в примерах.

**Задание.** Напишите консольную утилиту, которая является аналогом `ls`. Добавьте возможность ограничить выводимые файлы по размеру.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    raise NotImplementedError("Hello, Python")


def main(argv: list[str] | None = None) -> None:
    raise NotImplementedError("Hello, Python")


if __name__ == "__main__":
    raise NotImplementedError("Hello, Python")

**Задание.** Напишите утилиту-дразнилку, которая будет выводить указанное слово `N` раз. По умолчанию выводите `3` раза.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "word",
        help="the word to be repeated",
        metavar="WORD",
    )

    parser.add_argument(
        "-l",
        "--limit",
        default=3,
        help="the limit on the number of words",
        metavar="N",
        type=int
    )

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    ## YOUR CODE

if __name__ == "__main__":
    main(["python"])

python
python
python


In [ ]:
parse_args(["python"])

Namespace(word='python', limit=3)

In [ ]:
main(["python", "-l", "5"])

python
python
python
python
python


**Пример.** Ограничим слова. Теперь можно выводить либо `C++`, либо `Python`, либо `Rust`.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "word",
        choices=["C++", "Python", "Rust"],
        help="the word to be repeated",
        metavar="WORD",
    )

    parser.add_argument(
        "-l",
        "--limit",
        default=3,
        help="the limit on the number of words",
        metavar="N",
        type=int
    )

    return parser.parse_args(args)

In [ ]:
main(["Python"])

Python
Python
Python


In [ ]:
main(["C++"])

C++
C++
C++


**Задание.** Напишите консольную утилиту, которая считает сумму всех переданных чисел.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "numbers",
        help="the numbers to be summed up",
        metavar="NUM",
        nargs=argparse.REMAINDER,
        type=int,
    )

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    ## YOUR CODE


if __name__ == "__main__":
    main(["1", "2", "3", "4", "5", "6", "7"])

28


In [ ]:
parse_args(["1", "2", "3", "4", "5", "6", "7"])

Namespace(numbers=[1, 2, 3, 4, 5, 6, 7])

**Пояснение.** Атрибут `nargs` отвечает за число аргументов, которые должны быть получены. Можно указать число или маску (`?`, `*`, `+`).

**Задание.** Если необходимо, логировать начала суммирования.

In [ ]:
def parse_args(args: list[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "numbers",
        help="the numbers to be summed up",
        metavar="NUM",
        nargs="+",
        type=int,
    )

    parser.add_argument(
        "-d",
        "--debug",
        help="run in the debug mode",
        action="store_true",
    )

    return parser.parse_args(args)


def main(argv: list[str] | None = None) -> None:
    ## YOUR CODE

    if args.debug:
        print("Starting to sum up...")

    ## YOUR CODE


if __name__ == "__main__":
    main(["1", "2", "3", "4", "5", "6", "7", "--debug"])

Starting to sum up...
28


In [ ]:
main(["1", "2", "3", "4", "5", "6", "7"])

28


In [ ]:
parse_args(["1", "2", "3", "4", "5", "6", "7"])

Namespace(numbers=[1, 2, 3, 4, 5, 6, 7], debug=False)